# 💳 FLAN-T5 for Payment Intent and Slot Extraction
This notebook fine-tunes `google/flan-t5-base` to extract payment-related intents and slots from user commands like "Send $200 to Alice tomorrow".

In [ ]:
!pip install transformers datasets evaluate --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import random

In [ ]:
examples = [
    {
        'input': 'Send $200 to Alice tomorrow',
        'target': 'Intent: SendMoney | Slots: amount=$200, recipient=Alice, date=tomorrow'
    },
    {
        'input': 'Pay John 150 for dinner',
        'target': 'Intent: SendMoney | Slots: amount=150, recipient=John, reason=dinner'
    },
    {
        'input': 'Request $75 from Mike for groceries',
        'target': 'Intent: RequestMoney | Slots: amount=$75, sender=Mike, reason=groceries'
    },
    {
        'input': 'Remind me to pay rent on the 1st',
        'target': 'Intent: SetReminder | Slots: reason=rent, date=1st'
    }
]
dataset = Dataset.from_list(examples)

In [ ]:
model_name = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
def preprocess(example):
    model_input = tokenizer(example['input'], truncation=True, padding='max_length', max_length=64)
    labels = tokenizer(example['target'], truncation=True, padding='max_length', max_length=64)
    model_input['labels'] = labels['input_ids']
    return model_input

tokenized_dataset = dataset.map(preprocess)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='no',
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_steps=5,
    save_steps=10,
    save_total_limit=1
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)
trainer.train()

In [ ]:
input_text = 'Transfer 300 dollars to Mom next week'
inputs = tokenizer(input_text, return_tensors='pt')
outputs = model.generate(**inputs)
print('Prediction:', tokenizer.decode(outputs[0], skip_special_tokens=True))